# Project 4: West Nile Virus Prediction

## Problem Statement

## Executive Summary


### Contents:
- [Gather Information from Reddit](#Gather-Information-from-Reddit)
- [Modelling the Data](#Modelling-the-Data)
- [Finding Best Model](#Finding-Best-Model)

## Imports and Downloads

In [ ]:
#Imports:
import pandas as pd
from tqdm import tqdm
import requests
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [4]:
pip install geopy

    100% |████████████████████████████████| 102kB 177kB/s a 0:00:01
  Stored in directory: /Users/kokweilee/Library/Caches/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


## Train Data Cleaning and Engineering

In [ ]:
train = pd.read_csv('./predict-west-nile-virus/train.csv')
train.Date = pd.to_datetime(train.Date)

**Find Distances from the location to the weather stations**

Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level<br>
Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

In [205]:
import geopy.distance as gp

coord1 = (41.995,-87.933)  #coordinates for Chicago O'hare Airport (station 1)
coord2 = (41.786,-87.752)  #coordinates for Chicago Midway Airport (station 2)

dist_fr_1 = []
dist_fr_2 = []

for i in range(len(train.Latitude)):
    coord = (train.Latitude[i],train.Longitude[i])
    dist_fr_1.append(gp.distance(coord1, coord).km)
    dist_fr_2.append(gp.distance(coord2, coord).km)

train['dist_fr_1'] = dist_fr_1
train['dist_fr_2'] = dist_fr_2

In [206]:
train = train.drop(['Address','Block','Street','AddressNumberAndStreet','Latitude','Longitude'],axis=1)
train.head()

,Date,Species,Trap,AddressAccuracy,NumMosquitos,WnvPresent,dist_fr_1,dist_fr_2
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,9,1,0,11.822004,19.172879
1,2007-05-29,CULEX RESTUANS,T002,9,1,0,11.822004,19.172879
2,2007-05-29,CULEX RESTUANS,T007,9,1,0,13.565470,23.257125
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,8,1,0,9.261595,21.747902
4,2007-05-29,CULEX RESTUANS,T015,8,4,0,9.261595,21.747902


## Reading and Data Cleaning for Weather Data

In [229]:
weather = pd.read_csv('./predict-west-nile-virus/weather.csv')
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [230]:
weather.iloc[[87, 1745, 2067], 21] = (7.48, 7.61, 6.37) # AvgSpeed
weather.iloc[[87, 848, 2410, 2411], 17] = (29.37, 29.09, 29.30, 29.30) # StnPressure
weather.iloc[[848, 2410, 2412, 2415], 7] = (71.89, 64.67, 64.67, 59.44) # WetBulb

In [231]:
def timediff(a,b):
    amin = int(a[:-2])*60 + int(a[-2:])
    bmin = int(b[:-2])*60 + int(b[-2:])
    return abs(amin-bmin)

daylen = []
for i in range(0,len(weather.Sunrise),2):
    diff = timediff(weather.Sunset[i],weather.Sunrise[i])
    daylen.append(diff)
    daylen.append(diff)
weather['daylen']=daylen

In [232]:
weather.PrecipTotal = weather.PrecipTotal.replace('  T',0.001)
weather.PrecipTotal = weather.PrecipTotal.replace('M',0)

In [233]:
for i in range(len(weather.Tavg)):
    if weather.Tavg[i]=='M':
        weather.Tavg[i]= round((weather.Tmax[i]+weather.Tmin[i])/2)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [234]:
for index, row in weather[weather['SeaLevel']=='M'].iterrows():
    weather.SeaLevel[index] = np.nanmean(pd.to_numeric(weather[weather['Date']==row['Date']]['SeaLevel'], 
                                                          errors='coerce'))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [235]:
cols = ["+FC", "FC", "TS", "GR", "RA","DZ","SN","SG","GS","PL","IC","FG+","FG","BR","UP","HZ","FU",
"VA","DU","DS","PO","SA","SS","PY","SQ","DR","SH","FZ","MI","PR","BC","BL","VC"]

for col in cols:
    weather[col] = 0
    
weather.CodeSum = weather.CodeSum.str.rsplit()
for i in weather[cols]:
    for index, word in weather.CodeSum.iteritems():
        for code in word:
            if code == i:
                weather.loc[index, i] = 1

In [236]:
weather = weather.drop(['Depart','Heat','Cool','Depth','Water1','SnowFall','Sunrise','Sunset','CodeSum'],axis=1)
weather.Date = pd.to_datetime(weather.Date)
for col in ['Tavg','WetBulb','PrecipTotal','StnPressure','SeaLevel','AvgSpeed']:
    weather[col] = pd.to_numeric(weather[col])
weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,SeaLevel,...,PY,SQ,DR,SH,FZ,MI,PR,BC,BL,VC
0,1,2007-05-01,83,50,67.0,51,56.0,0.0,29.10,29.82,...,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-01,84,52,68.0,51,57.0,0.0,29.18,29.82,...,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-02,59,42,51.0,42,47.0,0.0,29.38,30.09,...,0,0,0,0,0,0,0,0,0,0
3,2,2007-05-02,60,43,52.0,42,47.0,0.0,29.44,30.08,...,0,0,0,0,0,0,0,0,0,0
4,1,2007-05-03,66,46,56.0,40,48.0,0.0,29.39,30.12,...,0,0,0,0,0,0,0,0,0,0


## Combine Train and Weather Data

In [237]:
#initialize new weather dataframe

wdf = pd.DataFrame()
for i in range(len(train.Date)):
    dateweather = weather[weather.Date == train.Date[i]].drop(['Date','Station'],axis=1)
    weightedw = (dist_fr_2[i] * dateweather.iloc[0].astype(float) +
             dist_fr_1[i] * dateweather.iloc[1].astype(float))/(dist_fr_1[i] + dist_fr_2[i])
    wdf[i] = weightedw
wdf = wdf.T

In [266]:
finaltrain = train.join(wdf)
finaltrain

,Date,Species,Trap,AddressAccuracy,NumMosquitos,WnvPresent,dist_fr_1,dist_fr_2,Tmax,Tmin,...,PY,SQ,DR,SH,FZ,MI,PR,BC,BL,VC
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,9,1,0,11.822004,19.172879,88.000000,61.907090,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2007-05-29,CULEX RESTUANS,T002,9,1,0,11.822004,19.172879,88.000000,61.907090,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007-05-29,CULEX RESTUANS,T007,9,1,0,13.565470,23.257125,88.000000,61.842004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,8,1,0,9.261595,21.747902,88.000000,61.493348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2007-05-29,CULEX RESTUANS,T015,8,4,0,9.261595,21.747902,88.000000,61.493348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2007-05-29,CULEX RESTUANS,T045,8,2,0,23.553811,16.652337,88.000000,62.929130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2007-05-29,CULEX RESTUANS,T046,8,1,0,25.817125,14.209564,88.000000,63.224989,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2007-05-29,CULEX PIPIENS/RESTUANS,T048,8,1,0,27.136705,12.129938,88.000000,63.455440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2007-05-29,CULEX RESTUANS,T048,8,2,0,27.136705,12.129938,88.000000,63.455440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2007-05-29,CULEX RESTUANS,T049,8,1,0,25.509596,14.650453,88.000000,63.175992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# do i want to remove rows? finaltrain[finaltrain.NumMosquitos==50]

## Test Data Cleaning and Engineering

In [249]:
test = pd.read_csv('./predict-west-nile-virus/test.csv')
test = test.drop(['Address','Block','Street','Trap','AddressNumberAndStreet'],axis=1)

In [251]:
coord1 = (41.995,-87.933)  #coordinates for Chicago O'hare Airport (station 1)
coord2 = (41.786,-87.752)  #coordinates for Chicago Midway Airport (station 2)

dist_fr_1 = []
dist_fr_2 = []

for i in range(len(test.Latitude)):
    coord = (test.Latitude[i],test.Longitude[i])
    dist_fr_1.append(gp.distance(coord1, coord).km)
    dist_fr_2.append(gp.distance(coord2, coord).km)

test['dist_fr_1'] = dist_fr_1
test['dist_fr_2'] = dist_fr_2

In [252]:
test.head()

,Id,Date,Species,Latitude,Longitude,AddressAccuracy,dist_fr_1,dist_fr_2
0,1,2008-06-11,CULEX PIPIENS/RESTUANS,41.95469,-87.800991,9,11.822004,19.172879
1,2,2008-06-11,CULEX RESTUANS,41.95469,-87.800991,9,11.822004,19.172879
2,3,2008-06-11,CULEX PIPIENS,41.95469,-87.800991,9,11.822004,19.172879
3,4,2008-06-11,CULEX SALINARIUS,41.95469,-87.800991,9,11.822004,19.172879
4,5,2008-06-11,CULEX TERRITANS,41.95469,-87.800991,9,11.822004,19.172879


## Combine Test and Weather Data

In [256]:
#initialize new weather dataframe

wdftest = pd.DataFrame()
for i in range(len(test.Date)):
    dateweather = weather[weather.Date == test.Date[i]].drop(['Date','Station'],axis=1)
    weightedw = (test.dist_fr_2[i] * dateweather.iloc[0].astype(float) +
             test.dist_fr_1[i] * dateweather.iloc[1].astype(float))/(test.dist_fr_1[i] + test.dist_fr_2[i])
    wdftest[i] = weightedw
wdftest = wdftest.T

In [261]:
finaltest = test.join(wdftest)
finaltest.drop(['Id','Latitude','Longitude'],axis=1,inplace=True)

#Test data source set to be 0.
finaltest['Source']=0
finaltest.columns

Index(['Date', 'Species', 'AddressAccuracy', 'dist_fr_1', 'dist_fr_2', 'Tmax',
       'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'PrecipTotal', 'StnPressure',
       'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 'daylen', '+FC',
       'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 'FG',
       'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ',
       'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC', 'Source'],
      dtype='object')

## Combine Train and Test Data to Perform Encoding

In [267]:
target = finaltrain.WnvPresent
finaltrain = finaltrain.drop(['Trap','NumMosquitos','WnvPresent'],axis=1)

#Train data source set to be 1.
finaltrain['Source']=1
finaltrain.columns

Index(['Date', 'Species', 'AddressAccuracy', 'dist_fr_1', 'dist_fr_2', 'Tmax',
       'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'PrecipTotal', 'StnPressure',
       'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 'daylen', '+FC',
       'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 'FG',
       'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ',
       'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC', 'Source'],
      dtype='object')

In [294]:
final = pd.concat([finaltrain, finaltest], ignore_index=True)
final = pd.concat([final, pd.get_dummies(final.Species)], axis = 1)
final.drop(['Species','Date'],axis=1,inplace=True)

## Split back into Train and Test Data

In [312]:
final_test = final[final.Source==0].reset_index().drop(['Source','index'],axis=1)
final_train = final[final.Source==1].drop(['Source'],axis=1)

# Run Classfication Models to Predict if WNV is Present

In [289]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (BaggingClassifier, RandomForestClassifier, AdaBoostClassifier)
from sklearn.svm import SVC

In [297]:
X_train, X_test, y_train, y_test = train_test_split(final_train, target, test_size=0.25, random_state=42)

In [304]:
1-target.mean()

0.9475537787930707

In [300]:
lr = LogisticRegression().fit(X_train,y_train)
print('score:', lr.score(X_test,y_test))
tn, fp, fn, tp = confusion_matrix(y_test, lr.predict(X_test)).ravel()
print('True Negative:',tn)
print('False Positive:',fp)
print('False Negative:',fn)
print('True Positive:',tp)

score: 0.9489912447658927
True Negative: 2493
False Positive: 0
False Negative: 134
True Positive: 0


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [301]:
knn = KNeighborsClassifier().fit(X_train,y_train)
print('score:', knn.score(X_test,y_test))
tn, fp, fn, tp = confusion_matrix(y_test, knn.predict(X_test)).ravel()
print('True Negative:',tn)
print('False Positive:',fp)
print('False Negative:',fn)
print('True Positive:',tp)

score: 0.9459459459459459
True Negative: 2474
False Positive: 19
False Negative: 123
True Positive: 11


In [302]:
dtc = DecisionTreeClassifier().fit(X_train,y_train)
print('score:',dtc.score(X_test,y_test))
tn, fp, fn, tp = confusion_matrix(y_test,dtc.predict(X_test)).ravel()
print('True Negative:',tn)
print('False Positive:',fp)
print('False Negative:',fn)
print('True Positive:',tp)

score: 0.9269128283212791
True Negative: 2415
False Positive: 78
False Negative: 114
True Positive: 20


In [305]:
bc = BaggingClassifier().fit(X_train,y_train)
print('score:', bc.score(X_test,y_test))
tn, fp, fn, tp = confusion_matrix(y_test, bc.predict(X_test)).ravel()
print('True Negative:',tn)
print('False Positive:',fp)
print('False Negative:',fn)
print('True Positive:',tp)

score: 0.9417586600685193
True Negative: 2455
False Positive: 38
False Negative: 115
True Positive: 19


In [306]:
rf = RandomForestClassifier().fit(X_train,y_train)
print('score:',rf.score(X_test,y_test))
tn, fp, fn, tp = confusion_matrix(y_test, rf.predict(X_test)).ravel()
print('True Negative:',tn)
print('False Positive:',fp)
print('False Negative:',fn)
print('True Positive:',tp)

score: 0.9429006471259992
True Negative: 2458
False Positive: 35
False Negative: 115
True Positive: 19


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [308]:
ab = AdaBoostClassifier().fit(X_train,y_train)
print('score:', ab.score(X_test,y_test))
tn, fp, fn, tp = confusion_matrix(y_test, ab.predict(X_test)).ravel()
print('True Negative:',tn)
print('False Positive:',fp)
print('False Negative:',fn)
print('True Positive:',tp)

score: 0.9489912447658927
True Negative: 2493
False Positive: 0
False Negative: 134
True Positive: 0


In [309]:
svc = SVC().fit(X_train,y_train)
print('score:', svc.score(X_test,y_test))
tn, fp, fn, tp = confusion_matrix(y_test, svc.predict(X_test)).ravel()
print('True Negative:',tn)
print('False Positive:',fp)
print('False Negative:',fn)
print('True Positive:',tp)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


score: 0.9486105824133994
True Negative: 2487
False Positive: 6
False Negative: 129
True Positive: 5


In [336]:
final_test

,AddressAccuracy,dist_fr_1,dist_fr_2,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,...,BL,VC,CULEX ERRATICUS,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,UNSPECIFIED CULEX
0,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,0,0,1,0,0,0,0,0
1,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,0,0,0,1,0,0,0,0
2,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,0,1,0,0,0,0,0,0
3,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,0,0,0,0,1,0,0,0
4,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,0,0,0,0,0,0,1,0
5,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,0,0,0,0,0,1,0,0
6,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,0,0,0,0,0,0,0,1
7,9,11.822004,19.172879,86.000000,62.907090,74.762836,55.618582,64.000000,0.000000,29.302885,...,0.0,0.0,1,0,0,0,0,0,0,0
8,9,13.565470,23.257125,86.000000,62.842004,74.736801,55.631599,64.000000,0.000000,29.302104,...,0.0,0.0,0,0,1,0,0,0,0,0
9,9,13.565470,23.257125,86.000000,62.842004,74.736801,55.631599,64.000000,0.000000,29.302104,...,0.0,0.0,0,0,0,1,0,0,0,0


In [337]:
predictions = rf.predict(final_test)

In [338]:
result = pd.DataFrame(np.array(range(1,len(predictions)+1)),columns=['Id'])
result['WnvPresent'] = predictions

In [339]:
result.to_csv('./results_rf.csv',index=False)

In [341]:
final_train.to_csv('./finaltrain.csv')

# Conclusion